# Load_confounds Tutorial


Download these packeges if you don't have them in you virtual envioroment

In [1]:
# Download these packeges if you don't have them in you virtual envioroment. (Un-comment code)

import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install nibabel
# !{sys.executable} -m pip install nilearn
# !{sys.executable} -m pip install pandas

Import needed moduels 

In [2]:
import pandas as pd
import nibabel as nib
from nilearn.maskers import NiftiMasker
from nilearn import image as nimg
from nilearn import masking

For using load confounds you have to load an fmriprep output functionl file. For most purpuses, if <br/>
you are not intrested in using the aroma confounds non-aggressivley, you should use the fmriprep <br/>
output file that ends with the suffix **"desc-preproc_bold.nii.gz"**


In [3]:
#Load the path of the functional file according to you enviroment's path
func_file =f"fmriprep/derivative-sub-Y01/sub-Y01/ses-1/func/sub-Y01_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

#Load the fmpriprep mask img
brain_mask = f"fmriprep/derivative-sub-Y01/sub-Y01/anat/sub-Y01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

## How to use use load_confounds

#### **fMRIPrep directories**

load_confounds extracts the relevant confounds from the fMRIPrep tsv output file that ends with the suffix **"confounds_timeseries.tsv"**. <br/> 
**Make sure that the image file, confound tsv and json files are in the same directory with BIDS-compliant names.** <br/>
Read more about fmriprep generated confounds from [fmriprep output info page](https://fmriprep.org/en/stable/outputs.html#confounds).<br/>

#### **A flexible approach versus a predefined approach** 

*nilearn.interfaces.fmriprep.**load_confounds*** function provides flexible parameters to retrieve the relevant columns from the TSV file generated by fMRIPrep. <br/>
*nilearn.interfaces.fmriprep.**load_confounds_strategy*** provides an interface to select confounds based on past literature with limited parameters for user customisation. <br/>
Read more at [load_confounds documentation](https://nilearn.github.io/dev/modules/generated/nilearn.interfaces.fmriprep.load_confounds.html), [load_confounds_strategy documentation](https://nilearn.github.io/dev/modules/generated/nilearn.interfaces.fmriprep.load_confounds_strategy.html#nilearn.interfaces.fmriprep.load_confounds_strategy).

### **Load confounds from file using a flexible strategy**

load_confound returns a confounds dataframe and a sample mask.<br/>
The confounds dataframe includes the regressors (confound colomns) selected with the spesefied strategy when calling the funnction.<br/>
Th sample mask is a mask for the non-study state volumes. This attribute should be passed to parameter sample_mask of nilearn.maskers.NiftiMasker or nilearn.signal.clean.



#### **Load_confounds parameters**

load_confounds(img_files, strategy=('motion', 'high_pass', 'wm_csf'), motion='full', scrub=5, fd_threshold=0.2, std_dvars_threshold=3,
wm_csf='basic', global_signal='basic', compcor='anat_combined', n_compcor='all', ica_aroma='full', demean=True)

**strategy** = the type of noise components to include from: “motion”, “high_pass”, “wm_csf”, “global_signal”, “compcor”, “scrub” .<br/>
For each component above, associated parameters will be applied if specified. If associated parameters are not specified, any values supplied to the parameters are ignored. <br/>
For example, if we specify that we want to include motion parameters, we need specify assosiated paramters according to the following:
<ul>
  <li>“basic” translation/rotation (6 parameters)</li>
  <li> “power2” translation/rotation + quadratic terms (12 parameters)</li>
  <li>“derivatives” translation/rotation + derivatives (12 parameters)</li>
  <li>“full” translation/rotation + derivatives + quadratic terms + power2d derivatives (24 parameters)</li>
</ul>

It is highly recommended to visit the [nilearn documentaion](https://nilearn.github.io/dev/modules/generated/nilearn.interfaces.fmriprep.load_confounds.html) for a detailed discription of all parameters.
    


##### **An important note on high_pass**
If your analysis includes separate high-pass filtering, do not include high_pass (cosine_XX) regressors.<br/>
**However**, CompCor and ica_aroma confounds were generated with the high_pass components of fMRIprep.<br/> **Therefore**, when using CompCor and ica_aroma  regressors, the corresponding cosine_XX regressors should also be included in the design matrix.

### An implemintation example

The following strategy removes motion confounds (using the "full" option), white matter signal (using the "basic" option), cerebrospinal fluid signal (basic). <br/>
high_pass regressors will not be included since we are going to include high_pass filtering later.

In [4]:
#Load the module that allows for flexible paramters strategy 
from nilearn.interfaces.fmriprep import load_confounds

#Extract confounds from fmripreps tsv confounds file
confounds_flexible, sample_mask_flexible = load_confounds(func_file, strategy=["motion", "wm_csf"],motion= "full", wm_csf="basic")

Take a look at the resultiing dataframe and make sure you have the right colomns

In [5]:
confounds_flexible.head()

,csf,rot_x,rot_x_derivative1,rot_x_derivative1_power2,rot_x_power2,rot_y,rot_y_derivative1,rot_y_derivative1_power2,rot_y_power2,rot_z,...,trans_x_power2,trans_y,trans_y_derivative1,trans_y_derivative1_power2,trans_y_power2,trans_z,trans_z_derivative1,trans_z_derivative1_power2,trans_z_power2,white_matter
0,4.328078,-0.005261,-0.000248,-1.838421e-07,-0.000051,-0.002360,-0.000120,-2.319602e-08,-0.000006,-0.001098,...,-0.001685,-0.120657,0.093807,-0.007659,-0.016965,-0.144648,0.039254,-0.000046,-0.021910,0.337259
1,0.407023,-0.005467,-0.000248,-1.838421e-07,-0.000051,-0.002472,-0.000120,-2.319602e-08,-0.000006,-0.001098,...,-0.001765,-0.025621,0.093807,-0.007659,-0.016158,-0.104915,0.039254,-0.000046,-0.022215,0.244897
2,-2.800916,-0.005291,0.000135,-1.952985e-07,-0.000051,-0.002360,0.000105,-2.335220e-08,-0.000006,-0.001098,...,-0.001766,-0.161594,-0.137201,0.001797,-0.011747,-0.137609,-0.033172,-0.000556,-0.022195,0.192182
3,-4.879048,-0.005258,-0.000008,-2.252468e-07,-0.000051,-0.002177,0.000177,-2.200909e-09,-0.000006,-0.001098,...,-0.001790,-0.094105,0.066261,-0.012136,-0.018558,-0.116703,0.020428,-0.001188,-0.022454,-0.073397
4,-3.560381,-0.005407,-0.000191,-2.040735e-07,-0.000051,-0.002177,-0.000007,-3.593521e-08,-0.000006,-0.001098,...,-0.001790,-0.054349,0.038528,-0.015111,-0.018307,-0.125721,-0.009497,-0.001543,-0.022449,0.808954


Use a NiftiMasker for fitting the model [nilearn.maskers.NiftiMasker](https://nilearn.github.io/dev/modules/generated/nilearn.maskers.NiftiMasker.html#nilearn.maskers.NiftiMasker)

In [6]:
#Masker is used for applying a mask to extract time-series from Niimg-like objects
## Not sure wether o standardize signal
masker= NiftiMasker(mask_img=brain_mask, standardize=True, memory='nilearn_cache', verbose=5, t_r=1.4, low_pass=0.1, high_pass=0.01)

#Extract time series and regress out confounds using GLM
time_series = masker.fit_transform(func_file, confounds=confounds_flexible, sample_mask=sample_mask_flexible)

# Transform the 2D data matrix back to an image in brain space
clean_img = masker.inverse_transform(time_series)

[NiftiMasker.fit] Loading data from None
[NiftiMasker.fit] Resampling mask
[Memory]0.1s, 0.0min    : Loading resample_img...
________________________________________resample_img cache loaded - 0.0s, 0.0min
[Memory]0.4s, 0.0min    : Loading _filter_and_mask...
____________________________________filter_and_mask cache loaded - 19.9s, 0.3min
[Memory]31.8s, 0.5min   : Loading unmask...
____________________________________________unmask cache loaded - 118.7s, 2.0min


Save your preprocced file 

In [7]:
nib.save(clean_img, f"tutorial/sub-01_ses-01_run-01_cleaned_flexible_option.nii.gz")

### **Load confounds from file using a  predefined strategy**

load_confounds_strategy outputs a confond data frame and a sample mask similiar to load_confounds' output.<br/>
**Available strategies:**  copied from [load_confounds_strategy documentation](https://nilearn.github.io/dev/modules/generated/nilearn.interfaces.fmriprep.load_confounds_strategy.html#nilearn.interfaces.fmriprep.load_confounds_strategy)<br/>
* **simple**: Load confounds for a simple denoising strategy commonly used in resting state functional connectivity, described in. With the global signal regression, this      approach can remove confounds without compromising the temporal degrees of freedom.
* **srubbing**: Load confounds for scrubbing. This approach can reliably remove the impact of high motion volumes in functional connectome, however, it might not be suitable   with subjects with high motion (more than 50% timeseries flagged as high motion). One should adjust the threshold based on the characteristics of the dataset, or remove high motion subjects from the dataset.
* **compcor**: Load confounds using the CompCor strategy. CompCor estimates noise through principal component analysis on regions that are unlikely to contain signal.  Thus it might not be a suitable approach for researchers who want explicit description of the source of noise. Empirically, Compcor has shown similar effect of removing           physiological noise as methods that explicitly model and remove physiology signals. Compcor can suffer from loss of temporal degrees of freedom when using explained               variance as the noise component estimation as the number of compcor component can be really high. Please refer to fMRIPrep documentation for more details.
* Load confounds for **non-aggresive ICA-AROMA** strategy. The strategy requires fMRIPrep outputs generated with –use-aroma suffixed with desc-                    smoothAROMAnonaggr_bold. ICA-AROMA increases the run time of fMRIPrep, however, the strategy performs well in various benchmarks. See Notes for more details about this option.


##### **An important note**
It is possible to overwrite the strategy's assosiated paramters. For example the in "simple" strategy the motion parameter is set to "full",
however, we can still use the "simple" strategy while choosing the "basic" selection of motion parameters.

### Implemintatig the "simple" strategy

In [8]:
from nilearn.interfaces.fmriprep import load_confounds_strategy
confounds_simple, sample_mask_simple = load_confounds_strategy(func_file, denoise_strategy="simple")

Take a look at the resultiing dataframe and make sure you have the right colomns

In [9]:
confounds_simple.head()

,cosine00,cosine01,cosine02,cosine03,cosine04,cosine05,cosine06,csf,rot_x,rot_x_derivative1,...,trans_x_power2,trans_y,trans_y_derivative1,trans_y_derivative1_power2,trans_y_power2,trans_z,trans_z_derivative1,trans_z_derivative1_power2,trans_z_power2,white_matter
0,0.072643,0.072641,0.072638,0.072633,0.072628,0.072621,0.072613,4.328078,-0.005261,-0.000248,...,-0.001685,-0.120657,0.093807,-0.007659,-0.016965,-0.144648,0.039254,-0.000046,-0.021910,0.337259
1,0.072638,0.072621,0.072593,0.072553,0.072503,0.072441,0.072368,0.407023,-0.005467,-0.000248,...,-0.001765,-0.025621,0.093807,-0.007659,-0.016158,-0.104915,0.039254,-0.000046,-0.022215,0.244897
2,0.072628,0.072581,0.072503,0.072394,0.072254,0.072082,0.071880,-2.800916,-0.005291,0.000135,...,-0.001766,-0.161594,-0.137201,0.001797,-0.011747,-0.137609,-0.033172,-0.000556,-0.022195,0.192182
3,0.072613,0.072521,0.072368,0.072155,0.071880,0.071545,0.071150,-4.879048,-0.005258,-0.000008,...,-0.001790,-0.094105,0.066261,-0.012136,-0.018558,-0.116703,0.020428,-0.001188,-0.022454,-0.073397
4,0.072593,0.072441,0.072189,0.071836,0.071383,0.070832,0.070181,-3.560381,-0.005407,-0.000191,...,-0.001790,-0.054349,0.038528,-0.015111,-0.018307,-0.125721,-0.009497,-0.001543,-0.022449,0.808954


In case you want to override the motion parameter and add a global_signal regressor you could use the folowing code:

In [10]:
confounds_simple_or, sample_mask_simple_or = load_confounds_strategy(func_file, denoise_strategy="simple", motion="basic", global_signal="basic")

Take a look at the resultiing dataframe, compare with dataframe above

In [11]:
confounds_simple_or.head()

,cosine00,cosine01,cosine02,cosine03,cosine04,cosine05,cosine06,csf,global_signal,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,white_matter
0,0.072643,0.072641,0.072638,0.072633,0.072628,0.072621,0.072613,4.328078,0.214747,-0.005261,-0.002360,-0.001098,0.023731,-0.120657,-0.144648,0.337259
1,0.072638,0.072621,0.072593,0.072553,0.072503,0.072441,0.072368,0.407023,-0.839065,-0.005467,-0.002472,-0.001098,0.028953,-0.025621,-0.104915,0.244897
2,0.072628,0.072581,0.072503,0.072394,0.072254,0.072082,0.071880,-2.800916,-1.666208,-0.005291,-0.002360,-0.001098,0.029118,-0.161594,-0.137609,0.192182
3,0.072613,0.072521,0.072368,0.072155,0.071880,0.071545,0.071150,-4.879048,-0.956079,-0.005258,-0.002177,-0.001098,0.033983,-0.094105,-0.116703,-0.073397
4,0.072593,0.072441,0.072189,0.071836,0.071383,0.070832,0.070181,-3.560381,-0.938975,-0.005407,-0.002177,-0.001098,0.033983,-0.054349,-0.125721,0.808954


In [ ]:
# Masker is used for applying a mask to extract time-series from Niimg-like objects
masker= NiftiMasker(mask_img=brain_mask, standardize=True, memory='nilearn_cache', verbose=5, t_r=1.4, low_pass=0.1, high_pass=0.01)

# Extract time series and regress out confounds using GLM
time_series = masker.fit_transform(func_file, confounds=confounds_simple, sample_mask=sample_mask_simple)

# Transform the 2D data matrix back to an image in brain space
clean_img = masker.inverse_transform(time_series)

[NiftiMasker.fit] Loading data from None
[NiftiMasker.fit] Resampling mask
[Memory]0.0s, 0.0min    : Loading resample_img...
________________________________________resample_img cache loaded - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling nilearn.maskers.nifti_masker._filter_and_mask...
_filter_and_mask('fmriprep/derivative-sub-Y01/sub-Y01/ses-1/func/sub-Y01_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', 
<nibabel.nifti1.Nifti1Image object at 0x7f5b6162b490>, { 'detrend': False,
  'dtype': None,
  'high_pass': 0.01,
  'high_variance_confounds': False,
  'low_pass': 0.1,
  'reports': True,
  'runs': None,
  'smoothing_fwhm': None,
  'standardize': True,
  'standardize_confounds': True,
  't_r': 1.4,
  'target_affine': None,
  'target_shape': None}, memory_level=1, memory=Memory(location=nilearn_cache/joblib), verbose=5, confounds=[      cosine00  cosine01  cosine02  cosine03  cosine04  cosine05 

Save your preprocced file 

In [ ]:
nib.save(clean_img, f"tutorial/sub-01_ses-01_run-01_cleaned_simple_strategy.nii.gz")